In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import Binarizer
import seaborn as sns

from sklearn.compose import make_column_transformer  # pour les pipelines composites
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.compose import make_column_selector  # pour transformer automatic des colonnes hétérogènes

from sklearn.pipeline import make_union


In [2]:
titanic = sns.load_dataset('titanic')
titanic.head()  # les données sont Hétérogène ( colonnes catégories et numériques-discrêtes-continues )

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
y = titanic['survived']  # on crée la target
X = titanic.drop('survived', axis=1) # le reste des données

In [4]:
#   model = make_pipeline(StandardScaler(), SGDClassifier()) = pb ici le normalisateur avec 'sex' idem pour l'estimateur sur les catégories
#   model.fit(X, y)  # -> ERROR de script donc

## Pb de données hétérogènes => Modèles composites avec transformateurs  -> make_column_transformer

### 1 - création d'un transformer manuellement ( on écrit les colonnes souhaitées ) puis 2 pipelines de preprocessing 
### et enfin 1 pipeline final pour model

In [5]:
# création de 2 catégories de colonnes ( 1 numérique et 1 catégorielle )  MAIS PB manuellement si beaucoup de colonnes !
numerical_features = ['pclass', 'age', 'fare']  
categorical_features = ['sex', 'deck', 'alone']

In [6]:
# création de 2 pipelines disctinctes pour ces 2 catégories de colonnes
numerical_pipeline = make_pipeline(SimpleImputer(), StandardScaler()) 
# SimpleImputer() remplace les NaN par la moyenne par défaut / StandardScaler = normalise les données
categorical_pipeline = make_pipeline(OneHotEncoder(), SimpleImputer(strategy = 'most_frequent'))
# ATTENTION : METTRE LE SIMPLEIMPUTER APRES LE ONEHOTENCODER SINON STRATEGY ERROR
# Stratégie d'imputer autre que moyenne car données = catégories
# OneHotEncoder pour transformer en matrice non ordonnée toutes les catégories de chaque colonne

In [7]:
# création de notre preprocessing final sur notre dataset 
preprocessor = make_column_transformer((numerical_pipeline, numerical_features), (categorical_pipeline, categorical_features))
# make_col_trans du module compose prend des tuples de (transformer, columns)

In [8]:
# création de notre model estimateur
model = make_pipeline(preprocessor, SGDClassifier())

In [9]:
# entrainement du model
model.fit(X, y)  # -> affiche le pipeline entrainé
""" Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['pclass', 'age', 'fare']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['sex', 'deck', 'alone'])])),
                ('sgdclassifier', SGDClassifier())])    """

" Pipeline(steps=[('columntransformer',\n                 ColumnTransformer(transformers=[('pipeline-1',\n                                                  Pipeline(steps=[('simpleimputer',\n                                                                   SimpleImputer()),\n                                                                  ('standardscaler',\n                                                                   StandardScaler())]),\n                                                  ['pclass', 'age', 'fare']),\n                                                 ('pipeline-2',\n                                                  Pipeline(steps=[('simpleimputer',\n                                                                   SimpleImputer(strategy='most_frequent')),\n                                                                  ('onehotencoder',\n                                                                   OneHotEncoder())]),\n                                    

### 2 - création automatique du transformer sur TOUTES les colonnes hétérogènes
### make_column_selector puis make_pipeline x 2 puis make_column_transformer

In [10]:
# création de 2 catégories automatiques de colonnes ( 1 numérique et 1 catégorielle )  si beaucoup de colonnes !
numerical_features = make_column_selector(dtype_include=np.number) # selection des colonnes de nombres
categorical_features = make_column_selector(dtype_exclude=np.number) # selection des colonnes excluant les nombres
# voir d'autres exemples sur le site sklearn d'include et exclude

In [11]:
# MEME CODE ENSUITE DE CI DESSUS

# création de 2 pipelines disctinctes pour ces 2 catégories de colonnes
numerical_pipeline = make_pipeline(SimpleImputer(), StandardScaler()) 
# SimpleImputer() remplace les NaN par la moyenne par défaut / StandardScaler = normalise les données
categorical_pipeline = make_pipeline( OneHotEncoder(), SimpleImputer(strategy = 'most_frequent'))
# Stratégie d'imputer autre que moyenne car données = catégories
# OneHotEncoder pour transformer en matrice non ordonnée toutes les catégories de chaque colonne

# création de notre preprocessing final sur notre dataset 
preprocessor = make_column_transformer((numerical_pipeline, numerical_features), (categorical_pipeline, categorical_features))
# make_col_trans du module compose prend des tuples de (transformer, columns)

# création de notre model estimateur
model = make_pipeline(preprocessor, SGDClassifier())

# entrainement du model
model.fit(X, y)  # -> affiche le pipeline entrainé

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001FF32D1E9A0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder()),
                                                                  ('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001FF32D1E3D0>)])),
                ('sgdclassifier', SGDClassifier())])

### 3 - Pipeline parallèles // avec make_union()

In [12]:
y = titanic['survived']  # on crée la target
X = titanic.drop('survived', axis=1) # le reste des données

In [13]:
numerical_features = X[['age', 'fare']]   # pour faire simple ici 'age' et 'fare' sont numeriques

In [14]:
numerical_features.describe  # il y a des Nan à traiter ensuite
type(numerical_features)

pandas.core.frame.DataFrame

In [15]:
# numerical_features = numerical_features.dropna(axis=0) # j'ai des pb à utiliser SimpleImpute pour les Nan

In [16]:
numerical_features.isna().sum()   # compte les nan par colonnes -> 177 dans 'age' et 0 dans 'fare'

age     177
fare      0
dtype: int64

In [17]:
replace_nan = SimpleImputer(missing_values=np.nan, strategy='mean', copy=False)
replace_nan.fit_transform(numerical_features)
# TRES IMPORTANT : le copy=False ne créé pas une copie où nan absents mais remplace le fichier original où nan absent => preuve ci-dessous

array([[22.        ,  7.25      ],
       [38.        , 71.2833    ],
       [26.        ,  7.925     ],
       ...,
       [29.69911765, 23.45      ],
       [26.        , 30.        ],
       [32.        ,  7.75      ]])

In [18]:
numerical_features.isna().sum()

age     0
fare    0
dtype: int64

In [19]:
pipeline = make_union(StandardScaler(), Binarizer())

In [20]:
pipeline.fit_transform(numerical_features)  # - > matrice (891, 4)  AUCUN LIGNE EFFACées !
# les 2 1eres colonnes sont celles Standardisées, les 2 dernières colonnes sont celles Binarisées ( discrétisées en 0 ou 1 )

array([[-0.5924806 , -0.50244517,  1.        ,  1.        ],
       [ 0.63878901,  0.78684529,  1.        ,  1.        ],
       [-0.2846632 , -0.48885426,  1.        ,  1.        ],
       ...,
       [ 0.        , -0.17626324,  1.        ,  1.        ],
       [-0.2846632 , -0.04438104,  1.        ,  1.        ],
       [ 0.17706291, -0.49237783,  1.        ,  1.        ]])